## PRENDRE EN COMPTE LES ERREURS D UPLOAD

In [1]:
import json
import time
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from collections import Counter
import pandas as pd
import numpy as np
import os
from PIL import Image
from collections import OrderedDict
import wordcloud

In [2]:
#Creation du blob de stockage des fichiers traites
# !az storage container create --name done --account-name vivatechstorage --account-key Mif4MhreZR9F0NkcIqAa9XQOUohHvb5JHTzlRhSEiLn4C44Xb21Orvpzs6pr+w/D+SivvMvJHNF2sCJKGyUD+g==

## 1 - Fonctions

In [2]:
def get_files_list():
    files = !az storage blob list --account-name vivatechstorage --account-key Mif4MhreZR9F0NkcIqAa9XQOUohHvb5JHTzlRhSEiLn4C44Xb21Orvpzs6pr+w/D+SivvMvJHNF2sCJKGyUD+g== -c video 
    files_json = json.loads(files.s)
    files_list = []
    for count, value in enumerate(files_json):
        files_list.append(value['name'])
    return files_list

In [3]:
def launch_job(iteration, file_name):
    #cree asset
    asset_name = file_name.partition(".")[0]
    #!az ams asset delete -n $asset_name -a vivatechams -g vivatech
    !az ams asset create -n $asset_name -a vivatechams -g vivatech
    print("#### asset " + asset_name + " created ####")
    
    #cree job
    job_name = "job_timestamp_" + str(iteration)
    file_url = "https://vivatechstorage.blob.core.windows.net/video/" + file_name
    #!az ams job delete --name $job_name --transform-name audioAnalyzerTransform  -a vivatechams -g vivatech 
    !az ams job start --name $job_name --transform-name audioAnalyzerTransform --files $file_url --output-asset-names $asset_name -a vivatechams -g vivatech --priority High
    print("####" + job_name + " launched ####")

In [4]:
def print_job_status(files_list):
    unfinished_jobs = len(files_list)
    #Waiting for jobs to finish
    while unfinished_jobs != 0:
        try:
            jobs = !az ams job list --account-name vivatechams -g vivatech --transform-name audioAnalyzerTransform
            jobs_json = json.loads(jobs.s)
            finished_jobs = 0
            scheduled_jobs = 0
            error_jobs = 0
            processing_jobs = 0
            queued_jobs = 0
            status = []
            for count, job in enumerate(jobs_json):
                if jobs_json[count]['outputs'][0]['state'] == "Finished":finished_jobs += 1
                status.append((jobs_json[count]["input"]['files'][0].partition("video/")[-1], ":", jobs_json[count]["outputs"][0]['progress'], '%', jobs_json[count]["outputs"][0]['state']))
            print(status , end="\r")
        except:print("Error catching status. Please wait...")
        if (jobs_json[count ]['outputs'][0]['state']=='Error') or (jobs_json[count ]['outputs'][0]['state']=='Canceled'):
            error_jobs+= 1
        if error_jobs > 0 :
            break
        unfinished_jobs = len(files_list) - finished_jobs
        #time.sleep(3)

In [5]:
def download_files():
    assets = !az ams asset list --account-name vivatechams -g vivatech
    assets_json = json.loads(assets.s)
    for count, asset in enumerate(assets_json):
        asset_id = "asset-" + assets_json[count]["assetId"]
        asset_name = assets_json[count]["name"] + ".json"
        !az storage blob download -n insights.json -f $asset_name -c $asset_id --account-name vivatechstorage

In [6]:
def delete_jobs():
    jobs = !az ams job list --account-name vivatechams -g vivatech --transform-name audioAnalyzerTransform
    jobs_json = json.loads(jobs.s)

    for count, job in enumerate(jobs_json):
        job_name = jobs_json[count]['name']
        !az ams job delete --name $job_name --transform-name audioAnalyzerTransform  -a vivatechams -g vivatech
        print(jobs_json[count]['name'], " deleted")

In [7]:
def delete_assets():
    assets = !az ams asset list --account-name vivatechams -g vivatech
    assets_json = json.loads(assets.s)
    for count, job in enumerate(assets_json):
        asset_name = assets_json[count]['name']
        !az ams asset delete --name $asset_name -a vivatechams -g vivatech
        print(assets_json[count]['name'], " deleted")

In [8]:
def extract_jsons_per_speaker(files_list):
    text_files = []
    text = ""
    for file_count, file in enumerate(files_list):
        with open(file.partition(".")[0] + ".json") as f:
            data = json.load(f)
            
        print(file)
        for count, sentence in enumerate(data['transcript']):
            text  = text + " " + data['transcript'][count]['text']

        text_files.append(text)
    
    return text_files

In [9]:
def extract_jsons(files_list):
    text_files = []
    text = ""
    for file_count, file in enumerate(files_list):
        with open(file.partition(".")[0] + ".json") as f:
            data = json.load(f)

        for count, sentence in enumerate(data['transcript']):
            text  = text + " " + data['transcript'][count]['text']

        #on sauvegarde dans la liste seulement si le fichier suivant correspond a une autre conference
        try:
            #if files_list[file_count][:-5] != files_list[file_count+1][:-5]:
            if files_list[file_count].partition("speaker")[0] != files_list[file_count+1].partition("speaker")[0]:
                text_files.append(text)
                text = ""
        except:text_files.append(text)
    
    return text_files

In [10]:
def get_words_count(text, file_name, path, granularity):
    tokenizer = RegexpTokenizer(r'\b[^\d\W]+\b')
    text_no_punc = tokenizer.tokenize(text)
    word_count_dict = Counter(w.lower() for w in text_no_punc if w.lower() not in stopwords.words('english'))
    #word_count_dict = word_count_dict.most_common()
    word_count_dict = dict(word_count_dict)
    word_count_df = pd.DataFrame.from_dict(word_count_dict, orient='index').reset_index()
    word_count_df.columns = ['word', 'count']
    word_count_df.sort_values(axis=0, by='count', ascending=False, inplace=True)
    word_count_df.reset_index(drop=True, inplace=True)
    
    half_day_indicator = ''
    #save
    if "_am_" in file_name: half_day_indicator = '_am'
    else: half_day_indicator = '_pm'
    
    word_count_df.to_excel("wordscount/"+ path + half_day_indicator +  '/wordscount_'+ granularity + '/' + file_name +".xlsx")
    
    return word_count_dict

In [11]:
class SimpleGroupedColorFunc(object):
    """Create a color function object which assigns EXACT colors
       to certain words based on the color to words mapping
       Parameters
       ----------
       color_to_words : dict(str -> list(str))
         A dictionary that maps a color to the list of words.
       default_color : str
         Color that will be assigned to a word that's not a member
         of any value from color_to_words.
    """

    def __init__(self, color_to_words, default_color):
        self.word_to_color = {word: color
                              for (color, words) in color_to_words.items()
                              for word in words}

        self.default_color = default_color

    def __call__(self, word, **kwargs):
        return self.word_to_color.get(word, self.default_color)

In [12]:
def generate_words_cloud(word_count_dict, file_name, mask, path):
    mask = np.array(Image.open("masks/" + mask))
    wc = WordCloud(background_color="rgba(255, 255, 255, 0)", mode="RGBA", mask=mask, 
                      font_path = "C:/Users/maxbauch/Box Sync/Vivatech/Tests nuages de mots/monserrat_font/Montserrat-Regular.otf", max_words=100) #, width=3000,height=2500 

    wc.generate_from_frequencies(word_count_dict)
    
    color_to_words = {
        '#FFFFFF': list(word_count_dict.keys())
    }
    default_color = 'white'

    # Create a color function with single tone
    grouped_color_func = SimpleGroupedColorFunc(color_to_words, default_color)

    wc.recolor(color_func=grouped_color_func)
    
    half_day_indicator = ''
    #save
    if "_am_" in file_name: half_day_indicator = '_am'
    else: half_day_indicator = '_pm'
    
    
    wc.to_file("wordclouds/" + path + half_day_indicator + '/' + file_name + '.png')

In [13]:
def get_set_files_name(files_list): return list(OrderedDict.fromkeys([x.partition("speaker")[0] for x in files_list]))
#     set_files_name = []
#     for count, file in enumerate(files_list):
#         set_files_name.append(files_list[count].partition("speaker")[0])
    
    #return list(set(set_files_name))

In [14]:
def copy_clean_video_container():
    key = "Mif4MhreZR9F0NkcIqAa9XQOUohHvb5JHTzlRhSEiLn4C44Xb21Orvpzs6pr+w/D+SivvMvJHNF2sCJKGyUD+g=="

    #copy
    !AzCopy /Source:https://vivatechstorage.blob.core.windows.net/video /Dest:https://vivatechstorage.blob.core.windows.net/done /SourceKey:$key /DestKey:$key /S

    #remove and create video container
    !az storage container delete --name video --account-name vivatechstorage --account-key Mif4MhreZR9F0NkcIqAa9XQOUohHvb5JHTzlRhSEiLn4C44Xb21Orvpzs6pr+w/D+SivvMvJHNF2sCJKGyUD+g==
    !az storage container create --name video --account-name vivatechstorage --account-key Mif4MhreZR9F0NkcIqAa9XQOUohHvb5JHTzlRhSEiLn4C44Xb21Orvpzs6pr+w/D+SivvMvJHNF2sCJKGyUD+g==

## 2 - Lancement des jobs

In [34]:
#obtenir la liste des fichiers
files_list = get_files_list()
#delete_jobs()
#delete_assets()
#Launch jobs
for count, file in enumerate(files_list):
    #print("#### LAUNCHING JOB " + str(count) + " ####")
    launch_job(count, files_list[count])

{
  "additionalProperties": {},
  "alternateId": null,
  "assetId": "b497430f-44b5-4008-b2d2-79a73a5bdad6",
  "container": null,
  "created": "2018-05-25T16:47:52.807000+00:00",
  "description": null,
  "id": "/subscriptions/1cb459f7-4b9d-4edf-8577-ee0e24fdf838/resourceGroups/vivatech/providers/Microsoft.Media/mediaservices/vivatechams/assets/25_pm_1630_1700_stage_one_ai_meets_gdpr_speaker_isabelle_falque_pierrotin",
  "lastModified": "2018-05-25T16:47:52.807000+00:00",
  "name": "25_pm_1630_1700_stage_one_ai_meets_gdpr_speaker_isabelle_falque_pierrotin",
  "resourceGroup": "vivatech",
  "storageAccountName": "vivatechstorage",
  "storageEncryptionFormat": "None",
  "type": "Microsoft.Media/mediaservices/assets"
}
#### asset 25_pm_1630_1700_stage_one_ai_meets_gdpr_speaker_isabelle_falque_pierrotin created ####
{
  "additionalProperties": {},
  "created": "2018-05-25T16:48:01.555718+00:00",
  "description": null,
  "id": "/subscriptions/1cb459f7-4b9d-4edf-8577-ee0e24fdf838/resourceGroup

In [35]:
#print job status
print_job_status(files_list)

In [36]:
#download files
download_files()

{
  "content": null,
  "deleted": false,
  "metadata": {},
  "name": "insights.json",
  "properties": {
    "appendBlobCommittedBlockCount": null,
    "blobTier": null,
    "blobTierChangeTime": null,
    "blobTierInferred": false,
    "blobType": "BlockBlob",
    "contentLength": 50510,
    "contentRange": "bytes 0-50509/50510",
    "contentSettings": {
      "cacheControl": null,
      "contentDisposition": null,
      "contentEncoding": null,
      "contentLanguage": null,
      "contentType": "application/json"
    },
    "copy": {
      "completionTime": null,
      "id": null,
      "progress": null,
      "source": null,
      "status": null,
      "statusDescription": null
    },
    "deletedTime": null,
    "etag": "\"0x8D5C260571E64B1\"",
    "lastModified": "2018-05-25T16:55:37+00:00",
    "lease": {
      "duration": null,
      "state": "available",
      "status": "unlocked"
    },
    "pageBlobSequenceNumber": null,
    "remainingRetentionDays": null,
    "serverEncrypte


Alive[################################################################]  100.0000%
Finished[#############################################################]  100.0000%


{
  "content": null,
  "deleted": false,
  "metadata": {},
  "name": "insights.json",
  "properties": {
    "appendBlobCommittedBlockCount": null,
    "blobTier": null,
    "blobTierChangeTime": null,
    "blobTierInferred": false,
    "blobType": "BlockBlob",
    "contentLength": 124434,
    "contentRange": "bytes 0-124433/124434",
    "contentSettings": {
      "cacheControl": null,
      "contentDisposition": null,
      "contentEncoding": null,
      "contentLanguage": null,
      "contentType": "application/json"
    },
    "copy": {
      "completionTime": null,
      "id": null,
      "progress": null,
      "source": null,
      "status": null,
      "statusDescription": null
    },
    "deletedTime": null,
    "etag": "\"0x8D5C260CD80551D\"",
    "lastModified": "2018-05-25T16:58:56+00:00",
    "lease": {
      "duration": null,
      "state": "available",
      "status": "unlocked"
    },
    "pageBlobSequenceNumber": null,
    "remainingRetentionDays": null,
    "serverEncry


Alive[################################################################]  100.0000%
Finished[#############################################################]  100.0000%


{
  "content": null,
  "deleted": false,
  "metadata": {},
  "name": "insights.json",
  "properties": {
    "appendBlobCommittedBlockCount": null,
    "blobTier": null,
    "blobTierChangeTime": null,
    "blobTierInferred": false,
    "blobType": "BlockBlob",
    "contentLength": 87884,
    "contentRange": "bytes 0-87883/87884",
    "contentSettings": {
      "cacheControl": null,
      "contentDisposition": null,
      "contentEncoding": null,
      "contentLanguage": null,
      "contentType": "application/json"
    },
    "copy": {
      "completionTime": null,
      "id": null,
      "progress": null,
      "source": null,
      "status": null,
      "statusDescription": null
    },
    "deletedTime": null,
    "etag": "\"0x8D5C26129CB4C47\"",
    "lastModified": "2018-05-25T17:01:31+00:00",
    "lease": {
      "duration": null,
      "state": "available",
      "status": "unlocked"
    },
    "pageBlobSequenceNumber": null,
    "remainingRetentionDays": null,
    "serverEncrypte


Alive[################################################################]  100.0000%
Finished[#############################################################]  100.0000%


In [37]:
#delete jobs
delete_jobs()

job_timestamp_0  deleted
job_timestamp_1  deleted
job_timestamp_2  deleted


In [38]:
#delete assets
delete_assets()

25_pm_1630_1700_stage_one_ai_meets_gdpr_speaker_isabelle_falque_pierrotin  deleted
25_pm_1645_1710_startup_stage_startup_survival_man_speaker_fleur_pellerin  deleted
25_pm_1700_1745_stage_one_startups_french_tech_speaker_mounir_mahjoubi  deleted


In [310]:
#copy_clean_video_container()

In [22]:
# files_list = ['24_am_1240_1240_stage_one_the_rise_of_the_inte_speaker_bill_mcdermott.mp3', '24_pm_1300_1400_stage_one_the_new_hospitality_speaker_sebastien_missoffe']

## 3 - Génération des viz

Extraction des données contenues dans les json

### 3.A - Bilan par speaker

Pas de nuages pour le bilan par speaker. Seulement un words count !

##### changer le path par demi-journee (24_am, 24_pm, 25_am, 25_pm)

In [39]:
half_day_path = "25"

In [40]:
# files_list.remove('25_pm_1620_1645_startup_stage_the_lure_of_the_usa_speaker_xavier _menchez.mp3')
# files_list.remove('25_pm_1510_1535_startup_stage_injecting_a_startup_speaker_chloe_bonnet.mp3')
# files_list.remove('25_pm_1400_1430_ceo_forum_a_digital_and_human_speaker_patricia_hollar.mp3')
# # # files_list.remove('24_pm_1700_1730_ceo_forum_pioneering_europes_speaker_herve_le_gavrian.mp3')
# # # files_list.remove('24_pm_1700_1730_ceo_forum_pioneering_europes_speaker_juliette_jarry.mp3')
# # # files_list.remove('24_pm_1735_1800_startup_stage_africas_horizon_eye_speaker_rania_belkahia.mp3')

In [41]:
files_list

['25_pm_1630_1700_stage_one_ai_meets_gdpr_speaker_isabelle_falque_pierrotin.mp3',
 '25_pm_1645_1710_startup_stage_startup_survival_man_speaker_fleur_pellerin.mp3',
 '25_pm_1700_1745_stage_one_startups_french_tech_speaker_mounir_mahjoubi.mp3']

In [42]:
text_per_speaker = extract_jsons_per_speaker(files_list)

25_pm_1630_1700_stage_one_ai_meets_gdpr_speaker_isabelle_falque_pierrotin.mp3
25_pm_1645_1710_startup_stage_startup_survival_man_speaker_fleur_pellerin.mp3
25_pm_1700_1745_stage_one_startups_french_tech_speaker_mounir_mahjoubi.mp3


In [43]:
#obtenir le words count des conférences
word_count_dicts_per_speaker = []
for count, text in enumerate(text_per_speaker):
    word_count_dicts_per_speaker.append(get_words_count(text, files_list[count], half_day_path, "par_speaker"))

### 3.B - Bilan par conférence

In [44]:
#extract jsons
texts = extract_jsons(files_list)

#### Préparer les masks de nuages

In [45]:
files_list

['25_pm_1630_1700_stage_one_ai_meets_gdpr_speaker_isabelle_falque_pierrotin.mp3',
 '25_pm_1645_1710_startup_stage_startup_survival_man_speaker_fleur_pellerin.mp3',
 '25_pm_1700_1745_stage_one_startups_french_tech_speaker_mounir_mahjoubi.mp3']

In [46]:
set_files_name = get_set_files_name(files_list)

In [47]:
set_files_name

['25_pm_1630_1700_stage_one_ai_meets_gdpr_',
 '25_pm_1645_1710_startup_stage_startup_survival_man_',
 '25_pm_1700_1745_stage_one_startups_french_tech_']

In [48]:
# masks = ["ball.png", "ball.png"] #africa
# len(set_files_name)==len(masks)

In [49]:
masks = []
for file in set_files_name:
    masks.append("ball.png")

#### Générer les words count et les nuages 

In [50]:
#obtenir le words count des conférences
word_count_dicts = []
for count, text in enumerate(texts):
    word_count_dicts.append(get_words_count(text, set_files_name[count], half_day_path, "par_conference"))
    generate_words_cloud(word_count_dicts[count], set_files_name[count],  masks[count], half_day_path)

### 3.C - Bilan de demi-journee

##### changer le path par demi-journee (24_am, 24_pm, 25_am, 25_pm) et le mask

In [51]:
half_day_path = "25_pm"
half_day_mask = "ball.png"

In [52]:
data = pd.DataFrame()
for file in os.listdir(os.getcwd() + "/wordscount/" + half_day_path + '/wordscount_par_conference'):
    print(os.getcwd() + "/wordscount/" + half_day_path + '/wordscount_par_conference/' + file)
    data = data.append(pd.read_excel(os.getcwd() + "/wordscount/" + half_day_path + '/wordscount_par_conference/' + file))

#generation fichier words count
data = data.groupby('word', as_index=False)['count'].sum()
data.sort_values(axis=0, by='count', ascending=False, inplace=True)
data.reset_index(drop=True, inplace=True)
half_day_file_name = half_day_path + '_bilan'
data.to_excel("wordscount/"+ half_day_path + '/' + half_day_file_name + ".xlsx")

#generation du nuage de mots
data_to_dict = data.set_index('word').to_dict()['count']
generate_words_cloud(data_to_dict, half_day_file_name,  half_day_mask, half_day_path[:-3])

C:\Users\maxbauch\Box Sync\Vivatech\script_prod/wordscount/25_pm/wordscount_par_conference/25_pm_1315_1400_stage_one_roundtable_equality_.xlsx
C:\Users\maxbauch\Box Sync\Vivatech\script_prod/wordscount/25_pm/wordscount_par_conference/25_pm_1400_1430_ceo_forum_a_digital_and_human_.xlsx
C:\Users\maxbauch\Box Sync\Vivatech\script_prod/wordscount/25_pm/wordscount_par_conference/25_pm_1405_1425_stage_x_tech_to_the_rescue_.xlsx
C:\Users\maxbauch\Box Sync\Vivatech\script_prod/wordscount/25_pm/wordscount_par_conference/25_pm_1405_1425_startup_stage_cryptocurrencies_and_.xlsx
C:\Users\maxbauch\Box Sync\Vivatech\script_prod/wordscount/25_pm/wordscount_par_conference/25_pm_1425_1435_stage_x_biased_algorithms_le_.xlsx
C:\Users\maxbauch\Box Sync\Vivatech\script_prod/wordscount/25_pm/wordscount_par_conference/25_pm_1425_1450_startup_stage_right_person_right_j_.xlsx
C:\Users\maxbauch\Box Sync\Vivatech\script_prod/wordscount/25_pm/wordscount_par_conference/25_pm_1430_1500_ceo_forum_the_startup_sector_

### 3.D - Bilan de journée

##### changer le path par demi-journee (24_am, 24_pm, 25_am, 25_pm) et le mask

In [53]:
half_day_path = "25_am"
half_day_mask = "ball.png"

In [54]:
data = pd.DataFrame()
for file in os.listdir(os.getcwd() + "/wordscount/" + half_day_path + '/wordscount_par_conference'):
    #print(os.getcwd() + "/wordscount/" + half_day_path + '/wordscount_par_conference/' + file)
    data = data.append(pd.read_excel(os.getcwd() + "/wordscount/" + half_day_path + '/wordscount_par_conference/' + file))

In [55]:
half_day_path = "25_pm"
half_day_mask = "ball.png"

In [56]:
for file in os.listdir(os.getcwd() + "/wordscount/" + half_day_path + '/wordscount_par_conference'):
    #print(os.getcwd() + "/wordscount/" + half_day_path + '/wordscount_par_conference/' + file)
    data = data.append(pd.read_excel(os.getcwd() + "/wordscount/" + half_day_path + '/wordscount_par_conference/' + file))

In [57]:
#generation fichier words count
data = data.groupby('word', as_index=False)['count'].sum()
data.sort_values(axis=0, by='count', ascending=False, inplace=True)
data.reset_index(drop=True, inplace=True)
half_day_file_name = half_day_path + '_bilan_journee'
data.to_excel("wordscount/"+ half_day_path + '/' + half_day_file_name + ".xlsx")

In [58]:
#generation du nuage de mots
data_to_dict = data.set_index('word').to_dict()['count']
generate_words_cloud(data_to_dict, half_day_file_name,  half_day_mask, half_day_path[:-3])